### Food Order Dataset
The "Food Delivery Order History Data" dataset on Kaggle provides a comprehensive collection of 21,321 food order records from various restaurants. It contains 29 columns detailing customer preferences, order trends, pricing, and delivery performance.

### My goal is to predict kpt_duration_minutes (Kitchen Preparation Time duration in minutes) from the "Food Delivery Order History Data" dataset. This is a regression problem, as kpt_duration_minutes is a continuous numerical variable.

#### This notebook contains EDA,NULL& Outlier Analysis,Column Transformation & Feature Design

In [30]:
import pandas as pd
import re
import numpy as np

In [31]:
food_history_df =pd.read_csv('/Users/sugashinikaliappan/Downloads/order_history_kaggle_data.csv')

In [32]:
food_history_df.shape

(21321, 29)

In [38]:
food_history_df.head(5)

,Restaurant ID,Restaurant name,Subzone,City,Order ID,Order Placed At,Order Status,Delivery,Distance,Items in order,...,Rating,Review,Cancellation / Rejection reason,Restaurant compensation (Cancellation),Restaurant penalty (Rejection),KPT duration (minutes),Rider wait time (minutes),Order Ready Marked,Customer complaint tag,Customer ID
0,20320607,Swaad,Sector 4,Delhi NCR,6168884918,"11:38 PM, September 10 2024",Delivered,Zomato Delivery,3km,"1 x Grilled Chicken Jamaican Tender, 1 x Grill...",...,NaN,NaN,NaN,NaN,NaN,18.35,11.6,Correctly,NaN,5d6c2b96db963098bc69768bea504c8bf46106a8a5178e...
1,20320607,Swaad,Sector 4,Delhi NCR,6170707559,"11:34 PM, September 10 2024",Delivered,Zomato Delivery,2km,"1 x Peri Peri Fries, 1 x Fried Chicken Angara ...",...,NaN,NaN,NaN,NaN,NaN,16.95,3.6,Correctly,NaN,0781815deb4a10a574e9fee4fa0b86b074d4a0b36175d5...
2,20320607,Swaad,Sector 4,Delhi NCR,6169375019,"03:52 PM, September 10 2024",Delivered,Zomato Delivery,<1km,1 x Bone in Peri Peri Grilled Chicken,...,NaN,NaN,NaN,NaN,NaN,14.05,12.2,Correctly,NaN,f93362f5ce5382657482d164e368186bcec9c6225fd93d...
3,20320607,Swaad,Sector 4,Delhi NCR,6151677434,"03:45 PM, September 10 2024",Delivered,Zomato Delivery,2km,"1 x Fried Chicken Ghostbuster Tender, 1 x Anga...",...,4.0,NaN,NaN,NaN,NaN,19.00,3.3,Correctly,NaN,1ed226d1b8a5f7acee12fc1d6676558330a3b2b742af5d...
4,20320607,Swaad,Sector 4,Delhi NCR,6167540897,"03:04 PM, September 10 2024",Delivered,Zomato Delivery,2km,"1 x Peri Peri Krispers, 1 x Fried Chicken Anga...",...,NaN,NaN,NaN,NaN,NaN,15.97,1.0,Correctly,NaN,d21a2ac6ea06b31cc3288ab20c4ef2f292066c096f2c5f...


### Refactoring Column Names

In [39]:
def clean_column_names(df):
    existing_columns = df.columns
    new_columns = [re.sub(r'[^a-zA-Z0-9_]', '_', col.replace(" ", "_").lower()) for col in existing_columns]
    new_columns = [re.sub(r'_{2,}', '_', col) for col in new_columns]
    new_columns = [col.rstrip('_') for col in new_columns]  # Remove trailing underscores
    df.columns = new_columns
    return df

food_history_df_after_renaming = clean_column_names(food_history_df)

### Null Values & Percentage at column level

In [40]:
null_values = food_history_df_after_renaming.isnull().sum()
null_values_percentage = (null_values / len(food_history_df_after_renaming)) * 100
null_columns_data = pd.concat([null_values, null_values_percentage], axis=1)
null_columns_data.columns = ['Null_Counts', 'Percentage']
null_columns_data['Percentage'] = null_columns_data['Percentage'].map('{:.2f}%'.format)  # Format percentage
print("\nNull Columns Percentage:")
display(null_columns_data)


Null Columns Percentage:


,Null_Counts,Percentage
restaurant_id,0,0.00%
restaurant_name,0,0.00%
subzone,0,0.00%
city,0,0.00%
order_id,0,0.00%
order_placed_at,0,0.00%
order_status,0,0.00%
delivery,0,0.00%
distance,0,0.00%
items_in_order,0,0.00%


### Analysis Summary
Instructions, Cancellation / Rejection reason, Restaurant compensation (Cancellation), Restaurant penalty (Rejection), Rating, Review, and Customer complaint tag, you should carefully consider whether they provide enough meaningful information to be used, even after imputation. You might decide to drop them entirely due to sparsity

### Handling target Null values

In [41]:
#NULL value analysis
display(food_history_df_after_renaming[food_history_df_after_renaming['kpt_duration_minutes'].isnull()].groupby(['order_status','order_ready_marked']).size().reset_index(name='null_kpt_duration_count'))

,order_status,order_ready_marked,null_kpt_duration_count
0,Delivered,Missed,197
1,Rejected,Missed,97
2,Timed out,Missed,1


In [28]:
food_history_df_after_renaming['kpt_duration_minutes'] = food_history_df_after_renaming.groupby(['order_status', 'order_ready_marked'])['kpt_duration_minutes'].transform(lambda x: x.fillna(x.median()))

# After group-wise fill, check if any NaNs remain (e.g., if a group was entirely NaN)
# If so, you might need a final global fill or decide to drop those remaining few.
if food_history_df_after_renaming['kpt_duration_minutes'].isna().sum() > 0:
    print("Some NaNs remain after group-wise fill. Filling globally.")
    global_median_kpt = food_history_df_after_renaming['kpt_duration_minutes'].median()
    food_history_df_after_renaming['kpt_duration_minutes'].fillna(global_median_kpt, inplace=True)

Some NaNs remain after group-wise fill. Filling globally.


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1216: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [42]:
display(food_history_df_after_renaming[food_history_df_after_renaming['kpt_duration_minutes'].isnull()].groupby(['order_status','order_ready_marked']).size().reset_index(name='null_kpt_duration_count'))

,order_status,order_ready_marked,null_kpt_duration_count
0,Delivered,Missed,197
1,Rejected,Missed,97
2,Timed out,Missed,1


In [50]:
suspicious_kpt_rows = food_history_df_after_renaming[
    (food_history_df_after_renaming['order_ready_marked'] == 'Missed') &
    (food_history_df_after_renaming['kpt_duration_minutes'].notnull())
].copy() 

print(f"Number of suspicious rows based on KPT and order_ready_marked: {len(suspicious_kpt_rows)}")

#conditions for anomalies based on KPT and item_count
anomaly_condition = (suspicious_kpt_rows['kpt_duration_minutes'] == 0) & (suspicious_kpt_rows['item_count'] >= 1)

high_kpt_small_item_condition = (
    (suspicious_kpt_rows['kpt_duration_minutes'] > 30) &
    (suspicious_kpt_rows['item_count'] == 1) &
    (suspicious_kpt_rows['item_categories'].apply(lambda x: any(cat in ['drinks', 'dip'] for cat in x)))
)

# Combine the anomaly conditions
anomalous_rows_to_drop_indices = suspicious_kpt_rows[
    anomaly_condition | high_kpt_small_item_condition
].index

print(f"Number of anomalous rows identified to drop: {len(anomalous_rows_to_drop_indices)}")

# Drop the identified anomalous rows from the main DataFrame
food_history_df_cleaned = food_history_df_after_renaming.drop(anomalous_rows_to_drop_indices)

print(f"\nOriginal DataFrame shape: {food_history_df_after_renaming.shape}")
print(f"\nCleaned DataFrame shape after dropping anomalies: {food_history_df_cleaned.shape}")

Number of suspicious rows based on KPT and order_ready_marked: 44
Number of anomalous rows identified to drop: 1

Original DataFrame shape: (21321, 42)

Cleaned DataFrame shape after dropping anomalies: (21320, 42)


### Columns Transformation

In [43]:
def clean_distance(distance_str):
    if pd.isna(distance_str):
        return np.nan
    elif '<' in distance_str:
        return 0.5  # <1km means approximately 0.5km
    else:
        # Extract the numeric part
        return float(distance_str.replace('km', ''))

food_history_df_after_renaming['distance'] = food_history_df_after_renaming['distance'].apply(clean_distance)

In [44]:
def extract_counts_and_items(items_str):
    """Extracts total item count, unique item count, and list of unique items."""
    if pd.isna(items_str) or items_str.strip() == "":
        return 0, 0, []

    # Find matches like "2 x itemA", "1 x itemB"
    matches = re.findall(r'(\d+)\s*x\s*([^,]+)', items_str, re.IGNORECASE)

    total_count = 0
    unique_items = []

    for quantity, item_name in matches:
        try:
            total_count += int(quantity)
            unique_items.append(item_name.strip())
        except ValueError:
            print(f"Warning: Could not convert '{quantity}' to an integer.")

    unique_item_count = len(unique_items)

    return total_count, unique_item_count, unique_items

# Apply and split into three columns
food_history_df_after_renaming[['item_count', 'unique_item_count', 'unique_items']] = food_history_df_after_renaming['items_in_order'].apply(
    lambda x: pd.Series(extract_counts_and_items(x))
)

In [47]:
def calculate_total_discount(row):
  """
  Calculates the total discount applied to an order, including all discount columns.

  Args:
    row: A row from the food_history_df_after_renaming DataFrame.

  Returns:
    The total discount amount.
  """
  total_discount = 0
  discount_columns = [
      'gold_discount',
      'restaurant_discount_promo',
      'restaurant_discount_flat_offs_freebies_others',
      'brand_pack_discount',
      'restaurant_compensation_cancellation' # Added this potential discount
  ]

  for col in discount_columns:
    if row[col] > 0:  # Check if the discount is greater than 0
        total_discount += row[col]

  return total_discount

# Apply the function to create a new 'total_discount' column
food_history_df_after_renaming['total_discount'] = food_history_df_after_renaming.apply(calculate_total_discount, axis=1)


### Creating some additional columns based on avaliable input columns which will impact kpt prediction

In [45]:
#calculating average cost per item
food_history_df_after_renaming['avg_cost_per_item'] = food_history_df_after_renaming['total'] / food_history_df_after_renaming['item_count']

In [48]:
#creating flags
food_history_df_after_renaming['discount_flag'] = np.where(food_history_df_after_renaming['bill_subtotal'] > food_history_df_after_renaming['total_discount'], 1, 0)
food_history_df_after_renaming['instructions_flag'] = np.where(food_history_df_after_renaming['instructions'].isnull(), 0, 1)

In [49]:
#creating labels for for category
vegetables = [
    'fries','paneer','krispers','rice','aloo','garlic','slaw','mushroom','potato','pepperoni','brocolli','veg','salsa','spinach','cous','onion','papad','soya'
]

non_vegetables = [
    'chicken','murgh','mutton','bacon','omelette','tangid','tangids','tenderloin',
]

drinks = [
    'lassi', 'buttermilk', 'lemonade', 'jal jeera', 'aam panna',
    'thandai', 'falooda', 'sharbat', 'coffee', 'tea', 'juice', 'soda',
    'coke', 'sprite', 'fanta', 'pepsi', 'mirinda','ale'
]
dip = ['sauce','dip','mayonnaise','mayo','ketchup'
]

all_unique_items = food_history_df_after_renaming['unique_items'].explode().unique()

def categorize_item(item):
    item_lower = item.lower()
    if any(veg_word in item_lower for veg_word in vegetables):
        return 'veg'
    elif any(non_veg_word in item_lower for non_veg_word in non_vegetables):
        return 'non veg'
    elif any(drink_word in item_lower for drink_word in drinks):
        return 'drinks'
    elif any(dip_word in item_lower for dip_word in dip):
        return 'dip'
    else:
        return 'other' # Or any other category for items not in the lists

# Create a dictionary mapping unique items to their categories
item_category_map = {item: categorize_item(item) for item in all_unique_items}

# Function to apply the categorization to the list of unique items in each row
def categorize_order_items(unique_items_list):
    if not unique_items_list:
        return []
    return [item_category_map.get(item, 'other') for item in unique_items_list]

# Apply the function to create a new column with item categories
food_history_df_after_renaming['item_categories'] = food_history_df_after_renaming['unique_items'].apply(categorize_order_items)

# You can optionally create flags for the presence of each category in an order
food_history_df_after_renaming['has_veg'] = food_history_df_after_renaming['item_categories'].apply(lambda x: 'veg' in x)
food_history_df_after_renaming['has_non_veg'] = food_history_df_after_renaming['item_categories'].apply(lambda x: 'non veg' in x)
food_history_df_after_renaming['has_drinks'] = food_history_df_after_renaming['item_categories'].apply(lambda x: 'drinks' in x)
food_history_df_after_renaming['has_dip'] = food_history_df_after_renaming['item_categories'].apply(lambda x: 'dip' in x)
food_history_df_after_renaming['has_other'] = food_history_df_after_renaming['item_categories'].apply(lambda x: 'other' in x)


print("\nDataFrame with Item Categories:")
display(food_history_df_after_renaming[['unique_items', 'item_categories', 'has_veg', 'has_non_veg', 'has_drinks', 'has_other','has_dip']].head())


DataFrame with Item Categories:


,unique_items,item_categories,has_veg,has_non_veg,has_drinks,has_other,has_dip
0,"[Grilled Chicken Jamaican Tender, Grilled Chic...","[non veg, non veg]",False,True,False,False,False
1,"[Peri Peri Fries, Fried Chicken Angara Tender,...","[veg, non veg, non veg]",True,True,False,False,False
2,[Bone in Peri Peri Grilled Chicken],[non veg],False,True,False,False,False
3,"[Fried Chicken Ghostbuster Tender, Angara Gril...","[non veg, veg]",True,True,False,False,False
4,"[Peri Peri Krispers, Fried Chicken Angara Tender]","[veg, non veg]",True,True,False,False,False


In [57]:
### Calculate look back window
food_history_df_after_renaming['order_placed_at'] = pd.to_datetime(food_history_df_after_renaming['order_placed_at'])

In [59]:
def calculate_time_window(order_timestamp, look_back_window):
    """
    Calculates the start and end timestamps for a historical look-back window
    relative to the order placed timestamp.

    Args:
        order_timestamp (pd.Timestamp): The timestamp when the order was placed.
        look_back_window (str): The duration of the look-back window ('current_day', '1_week').

    Returns:
        tuple: A tuple containing the start timestamp (T1) and the end timestamp (T2)
               for the historical window.
    """
    if look_back_window == 'current_day':
        # T1 is order_placed_timestamp - 90 minutes (buffer)
        T1 = order_timestamp - pd.Timedelta(minutes=90)
        # T2 is the start of the day of the order_placed_at
        T2 = pd.Timestamp(order_timestamp.date())
    elif look_back_window == '1_week':
        # T1 is the end of the day before the order_placed_at
        T1 = pd.Timestamp(order_timestamp.date()) - pd.Timedelta(seconds=1)
        # T2 is 7 days before the start of the day of T1
        T2 = pd.Timestamp(T1.date()) - pd.Timedelta(days=7)
    else:
        raise ValueError("Invalid look_back_window. Use 'current_day' or '1_week'.")

    # Ensure T2 is always before T1 (historical window is before the order)
    if T2 > T1:
        T1, T2 = T2, T1 # Swap if necessary

    return T2, T1 # Return in chronological order (start, end)


# Calculate window for 'current_day' for the first row
first_order_timestamp = food_history_df_after_renaming.loc[0, 'order_placed_at']
current_day_T2, current_day_T1 = calculate_time_window(first_order_timestamp, 'current_day')
print(f"For first order ({first_order_timestamp}):")
print(f"  'current_day' window: T2 = {current_day_T2}, T1 = {current_day_T1}")

# Calculate window for '1_week' for the first row
week_T2, week_T1 = calculate_time_window(first_order_timestamp, '1_week')
print(f"  '1_week' window: T2 = {week_T2}, T1 = {week_T1}")



For first order (2024-09-10 23:38:00):
  'current_day' window: T2 = 2024-09-10 00:00:00, T1 = 2024-09-10 22:08:00
  '1_week' window: T2 = 2024-09-02 00:00:00, T1 = 2024-09-09 23:59:59


In [63]:
def generate_historical_features(df, current_order_row, look_back_window='current_day', group_by_cols=['restaurant'], agg_col='kpt_duration_minutes', agg_func='avg'):
    """
    Generates historical features for a given order based on historical data.

    Args:
        df (pd.DataFrame): The full historical dataset.
        current_order_row (pd.Series): The row representing the current order
                                      for which features are being generated.
        look_back_window (str): The historical window to consider ('current_day' or '1_week').
        group_by_cols (list): List of column names to group by.
        agg_col (str): The key performance indicator column to aggregate (e.g., 'kpt_duration_minutes').
        agg_func (str): The aggregation function ('avg', 'median', '5th_percentile', etc.).

    Returns:
        float: The calculated historical feature value for the given order.
               Returns NaN if no historical data is found within the window and grouping.
    """
    order_timestamp = current_order_row['order_placed_at']
    T2, T1 = calculate_time_window(order_timestamp, look_back_window)

    # Filter historical data within the time window
    historical_data = df[
        (df['order_placed_at'] >= T2) &
        (df['order_placed_at'] <= T1)
    ].copy() # Use copy to avoid SettingWithCopyWarning

    # Apply additional reasonable filters (example: exclude the current order itself, completed orders)
    # We exclude the current order to prevent data leakage.
    historical_data = historical_data[historical_data.index != current_order_row.name]
    # Example: Filter for delivered orders (adjust based on domain knowledge)
    historical_data = historical_data[historical_data['order_status'] == 'Delivered']


    # Filter historical data based on the grouping columns matching the current order
    for col in group_by_cols:
        if col in current_order_row and col in historical_data.columns:
             historical_data = historical_data[historical_data[col] == current_order_row[col]]
        else:
             print(f"Warning: Grouping column '{col}' not found in current order row or historical data. Skipping group by on this column.")


    if historical_data.empty:
        return np.nan # Return NaN if no historical data matches the criteria

    # Perform aggregation
    if agg_func == 'avg':
        return historical_data[agg_col].mean()
    elif agg_func == 'median':
        return historical_data[agg_col].median()
    elif agg_func == '5th_percentile':
        return historical_data[agg_col].quantile(0.05)
    elif agg_func == '25th_percentile':
        return historical_data[agg_col].quantile(0.25)
    elif agg_func == '75th_percentile':
        return historical_data[agg_col].quantile(0.75)
    elif agg_func == '95th_percentile':
        return historical_data[agg_col].quantile(0.95)
    else:
        raise ValueError(f"Invalid aggregation function: {agg_func}")

# --- Example of how to apply this to generate features for the entire DataFrame ---

# Select a subset of the data for demonstration purposes due to computation time
# In a real scenario, you would apply this to the entire cleaned DataFrame.
df_sample = food_history_df_after_renaming.sample(n=1000, random_state=42).reset_index(drop=True)

# Define feature configurations
feature_configs = [
    {'name': 'avg_kpt_current_day_restaurant', 'window': 'current_day', 'group_by': ['restaurant_name'], 'agg_col': 'kpt_duration_minutes', 'agg_func': 'avg'},
    {'name': 'median_kpt_1_week_restaurant_zone', 'window': '1_week', 'group_by': ['restaurant_name', 'subzone'], 'agg_col': 'kpt_duration_minutes', 'agg_func': 'median'},
    {'name': 'avg_item_count_current_day_restaurant', 'window': 'current_day', 'group_by': ['restaurant_name'], 'agg_col': 'item_count', 'agg_func': 'avg'},
    {'name': 'median_bill_subtotal_1_week_zone', 'window': '1_week', 'group_by': ['subzone'], 'agg_col': 'bill_subtotal', 'agg_func': 'median'},
    {'name': 'avg_distance_current_day_restaurant', 'window': 'current_day', 'group_by': ['restaurant_name'], 'agg_col': 'distance', 'agg_func': 'avg'},
]

# Generate features for each row in the sample DataFrame
for config in feature_configs:
    feature_name = config['name']
    print(f"Generating feature: {feature_name}...")
    df_sample[feature_name] = df_sample.apply(
        lambda row: generate_historical_features(
            food_history_df_after_renaming, # Use the full cleaned DataFrame as the source for historical data
            row,
            look_back_window=config['window'],
            group_by_cols=config['group_by'],
            agg_col=config['agg_col'],
            agg_func=config['agg_func']
        ),
        axis=1
    )
    print(f"Finished generating feature: {feature_name}")

print("\nSample DataFrame with Historical Features:")
display(df_sample.head())



Generating feature: avg_kpt_current_day_restaurant...
Finished generating feature: avg_kpt_current_day_restaurant
Generating feature: median_kpt_1_week_restaurant_zone...
Finished generating feature: median_kpt_1_week_restaurant_zone
Generating feature: avg_item_count_current_day_restaurant...
Finished generating feature: avg_item_count_current_day_restaurant
Generating feature: median_bill_subtotal_1_week_zone...
Finished generating feature: median_bill_subtotal_1_week_zone
Generating feature: avg_distance_current_day_restaurant...
Finished generating feature: avg_distance_current_day_restaurant

Sample DataFrame with Historical Features:


,restaurant_id,restaurant_name,subzone,city,order_id,order_placed_at,order_status,delivery,distance,items_in_order,...,has_veg,has_non_veg,has_drinks,has_dip,has_other,avg_kpt_current_day_restaurant,median_kpt_1_week_restaurant_zone,avg_item_count_current_day_restaurant,median_bill_subtotal_1_week_zone,avg_distance_current_day_restaurant
0,20659868,Aura Pizzas,Sector 4,Delhi NCR,6455771118,2024-12-23 20:38:00,Delivered,Zomato Delivery,1.0,1 x Chilli Cheese Garlic Bread,...,True,False,False,False,False,16.986667,15.630,2.051282,745.0,4.256410
1,20882652,Aura Pizzas,DLF Phase 1,Delhi NCR,6251272062,2024-10-08 19:41:00,Delivered,Zomato Delivery,3.0,"1 x Bageecha Pizza, 1 x Mushroom Mozzarella Melt",...,True,False,False,False,True,14.971429,17.615,1.571429,599.0,4.535714
2,21173951,Aura Pizzas,Vasant Kunj,Delhi NCR,6307857338,2024-10-29 17:02:00,Delivered,Zomato Delivery,2.0,"1 x Chicken Pepperoni Pizza, 1 x Margherita Pizza",...,True,False,False,False,True,16.250476,15.470,2.071429,697.5,4.214286
3,20659868,Aura Pizzas,Sector 4,Delhi NCR,6210020988,2024-09-24 20:49:00,Delivered,Zomato Delivery,6.0,2 x Jamaican Chicken Melt,...,False,True,False,False,False,16.591236,14.885,2.244444,599.0,4.244444
4,21077127,Aura Pizzas,Sector 135,Delhi NCR,6436301172,2024-12-13 21:18:00,Delivered,Zomato Delivery,10.0,1 x Makhani Paneer Pizza,...,True,False,False,False,False,16.860656,16.270,2.196721,659.0,4.016393
